In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.models import Model

from dataset_utilits import load_imagesPath_ages_sex
from inception_utilits import stem_block, inception_a_block, inception_b_block, inception_c_block, reduction_a_block, \
    reduction_b_block

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
train_truth = pd.read_csv('./data/train/train.csv')
validation_truth = pd.read_csv('./data/validation/validation.csv')
test_truth = pd.read_excel('./data/test/test.xlsx')
IMG_SHAPE = (299, 299, 1)

In [3]:
path_data_dir = 'data'
full_path = os.path.join(os.getcwd(), path_data_dir)
X_train, age_train, sex_train, X_validation, age_validation, sex_validation, X_test, age_test, sex_test = load_imagesPath_ages_sex(
    full_path, train_truth, validation_truth, test_truth, None)

age_train = [x / 200. for x in age_train]
age_validation = [x / 200. for x in age_validation]
age_test = [x / 200. for x in age_test]

age_train_df = pd.DataFrame(age_train, columns=['age'])
age_validation_df = pd.DataFrame(age_validation, columns=['age'])
age_test_df = pd.DataFrame(age_test, columns=['age'])

age_train_df = age_train_df.astype('float32')
age_validation_df = age_validation_df.astype('float32')
age_test_df = age_test_df.astype('float32')

prendo le image_names da  C:\Users\Victor\Development\BDMA\UniPd\HDA\HDA-bone-age-prediction\data\train\reshaped
prendo le image_names da  C:\Users\Victor\Development\BDMA\UniPd\HDA\HDA-bone-age-prediction\data\validation\reshaped
prendo le image_names da  C:\Users\Victor\Development\BDMA\UniPd\HDA\HDA-bone-age-prediction\data\test\reshaped


In [5]:
print("X shape:", len(X_test))
print("X train shape:", len(X_train))
print("X val shape:", len(X_validation))

X shape: 200
X train shape: 12600
X val shape: 1424


In [6]:
X_test[:5]

['C:\\Users\\Victor\\Development\\BDMA\\UniPd\\HDA\\HDA-bone-age-prediction\\data\\test\\reshaped\\4360.png',
 'C:\\Users\\Victor\\Development\\BDMA\\UniPd\\HDA\\HDA-bone-age-prediction\\data\\test\\reshaped\\4361.png',
 'C:\\Users\\Victor\\Development\\BDMA\\UniPd\\HDA\\HDA-bone-age-prediction\\data\\test\\reshaped\\4362.png',
 'C:\\Users\\Victor\\Development\\BDMA\\UniPd\\HDA\\HDA-bone-age-prediction\\data\\test\\reshaped\\4363.png',
 'C:\\Users\\Victor\\Development\\BDMA\\UniPd\\HDA\\HDA-bone-age-prediction\\data\\test\\reshaped\\4364.png']

### Second branch data

In [7]:
train_features = pd.read_csv('./data/train/maxes.csv')
train_features['image'] = train_features['image'].str[:-4].astype(int)
second_branch_df_train = pd.merge(train_features, train_truth, left_on='image', right_on='id')
second_branch_df_train = second_branch_df_train.drop(['id', 'image', 'boneage'], axis=1).rename({'male': 'sex'}, axis=1)
second_branch_df_train['sex'] = second_branch_df_train['sex'].replace({True: 1, False: 0})

validation_features = pd.read_csv('./data/validation/maxes.csv')
validation_features['image'] = validation_features['image'].str[:-4].astype(int)
second_branch_df_val = pd.merge(validation_features, validation_truth, left_on='image', right_on='Image ID')
second_branch_df_val = second_branch_df_val.drop(['Image ID', 'image', 'Bone Age (months)'], axis=1).rename(
    {'male': 'sex'}, axis=1)
second_branch_df_val['sex'] = second_branch_df_val['sex'].replace({True: 1, False: 0})

test_features = pd.read_csv('./data/test/maxes.csv')
test_features['image'] = test_features['image'].str[:-4].astype(int)
second_branch_df_test = pd.merge(test_features, test_truth, left_on='image', right_on='Case ID')
second_branch_df_test = second_branch_df_test.drop(['Case ID', 'image', 'Ground truth bone age (months)'],
                                                   axis=1).rename({'Sex': 'sex'}, axis=1)
second_branch_df_test['sex'] = second_branch_df_test['sex'].replace({'M': 1, 'F': 0})

Try if the second branch works by itself:

In [8]:
# def create_dataset_branch2(second_branch_df, ages, batch_size, shuffle, cache_file=None):

#     # Create a Dataset object
#     dataset = tf.data.Dataset.from_tensor_slices((second_branch_df, ages))

#     # Cache dataset
#     if cache_file:
#         dataset = dataset.cache(cache_file)


#     # Shuffle
#     if shuffle:
#         dataset = dataset.shuffle(len(second_branch_df))

#     # Repeat the dataset indefinitely
#     dataset = dataset.repeat()

#     # Batch
#     dataset = dataset.batch(batch_size=batch_size)

#     # Prefetch
#     dataset = dataset.prefetch(buffer_size=1)

#     return dataset

# batch_size = 32
# train_dataset_branch_2 = create_dataset_branch2(second_branch_df = second_branch_df_train, 
#                     ages = age_train, 
#                     batch_size = batch_size, 
#                     shuffle = False )  

# validation_dataset_branch_2 = create_dataset_branch2(second_branch_df = second_branch_df_val, 
#                     ages = age_validation, 
#                     batch_size = batch_size, 
#                     shuffle = False ) 

# def dense_branch_2(X_input):
#     X = Dense(64, activation='relu', name='first_dense_branch_2')(X_input)
#     X = Dense(32, activation='relu', name='second_dense_branch_2')(X)
#     X = Flatten()(X)
#     # max_pooling o global_pooling (valore unico) [regolarizza vs overfitting] o dense più piccolo - > dense + pooling
#     return X


# def model_assembly_example(input_shape_dataset):
#     # branch 2
#     X_input_branch2 = Input(input_shape_dataset)
#     branch2 = dense_branch_2(X_input_branch2)
#     X = Dense(1, activation = 'relu', name='final')(branch2)


#     # Create model
#     model = Model(inputs = X_input_branch2, outputs = X, name='branch2_attempt')
#     return model

# train_steps = int(np.ceil(len(sex_train) / batch_size))
# val_steps = int(np.ceil(len(sex_validation) / batch_size))

# model = model_assembly_example(input_shape_dataset=5)

# model.compile(optimizer="adam", loss="mean_squared_error", metrics=tf.keras.metrics.mean_squared_error)

# # Create a callback for early stopping 
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# history = model.fit(train_dataset_branch_2, 
#                     validation_data = validation_dataset_branch_2, 
#                     epochs=100, 
#                     steps_per_epoch=train_steps,
#                     validation_steps=val_steps,
#                     callbacks=[callback])

## MODEL

In [9]:
def Inceptionv4(X_input):  #  (input_shape):
    """
    Implementation of the Inception-v4 architecture

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # # Define the input as a tensor with shape input_shape (1 line)
    # X_input = Input(input_shape)

    # Call the above functions for the stem, inception-a, reduction-a, inception-b, reduction-b and inception-c blocks
    X = stem_block(X_input)

    # Four Inception A blocks
    X = inception_a_block(X, 'a1')
    X = inception_a_block(X, 'a2')
    X = inception_a_block(X, 'a3')
    X = inception_a_block(X, 'a4')

    # Reduction A block
    X = reduction_a_block(X)

    # Seven Inception B blocks
    X = inception_b_block(X, 'b1')
    X = inception_b_block(X, 'b2')
    X = inception_b_block(X, 'b3')
    X = inception_b_block(X, 'b4')
    X = inception_b_block(X, 'b5')
    X = inception_b_block(X, 'b6')
    X = inception_b_block(X, 'b7')

    # Reduction B block
    X = reduction_b_block(X)

    # Three Inception C blocks
    X = inception_c_block(X, 'c1')
    X = inception_c_block(X, 'c2')
    X = inception_c_block(X, 'c3')

    # AVGPOOL (1 line). Use "X = AveragePooling2D(...)(X)"
    kernel_pooling = X.get_shape()[1:3]
    X = AveragePooling2D(kernel_pooling, name='avg_pool')(X)
    X = Flatten()(X)

    # Dropout
    X = Dropout(rate=0.2)(X)

    # Output layer
    # X = Dense(1, activation='relu', name='fc')(X)

    # # Create model
    # model = Model(inputs = X_input, outputs = X, name='Inceptionv4')

    # return model
    return X

In [10]:
def dense_branch_2(X_input):
    X = Dense(64, activation='relu', name='first_dense_branch_2')(X_input)
    X = Dense(32, activation='relu', name='second_dense_branch_2')(X)
    X = Flatten()(X)
    # max_pooling o global_pooling (valore unico) [regolarizza vs overfitting] o dense più piccolo - > dense + pooling
    return X


In [11]:
def process_image_2(path_df, label):
    # Desired size
    # num_row = IMG_SHAPE[1]
    # num_col = IMG_SHAPE[0]
    size = IMG_SHAPE[1]
    path, df = path_df

    # Get the image
    img = tf.io.read_file(path)
    # Decode the PNG
    img = tf.image.decode_png(img)
    # Resize image
    img = tf.image.resize(img, (size, size))
    # Reshape image (this is not necessary but I do it so that I don't need to be modifying the shape in the input layer)
    #img = tf.reshape(img, [size, size, 3])
    # Cast image to float32
    img = tf.cast(img, tf.float32)
    # Normalize image
    img = img / 255.0

    return (img, df), label

In [12]:
def create_dataset_images(img_paths, second_branch_df, ages, batch_size, shuffle, cache_file=None, repeat=True):
    # Create a Dataset object
    second_branch_df = second_branch_df.astype('float32')
    dataset = tf.data.Dataset.from_tensor_slices(((img_paths, second_branch_df), ages)).map(process_image_2)

    # Cache dataset
    if cache_file:
        dataset = dataset.cache(cache_file)

    # Shuffle
    if shuffle:
        dataset = dataset.shuffle(len(img_paths))

    # Repeat the dataset indefinitely
    if repeat:
        dataset = dataset.repeat()

    # Batch
    dataset = dataset.batch(batch_size=batch_size)

    # Prefetch
    dataset = dataset.prefetch(buffer_size=1)

    return dataset


batch_size = 8
train_dataset = create_dataset_images(img_paths=X_train,
                                      second_branch_df=second_branch_df_train[['sex']],  # Only gender
                                      # second_branch_df=second_branch_df_train,   # Full features
                                      ages=age_train_df,
                                      batch_size=batch_size,
                                      shuffle=False)

validation_dataset = create_dataset_images(img_paths=X_validation,
                                           second_branch_df=second_branch_df_val[['sex']],  # Only gender
                                           # second_branch_df=second_branch_df_val,  # Full features
                                           ages=age_validation_df,
                                           batch_size=batch_size,
                                           shuffle=False)

train_steps = int(np.ceil(len(X_train) / batch_size))
validation_steps = int(np.ceil(len(X_validation) / batch_size))

In [13]:
def model_assembly(input_shape_img, input_shape_dataset):
    # Branch 1
    X_input_branch1 = Input(input_shape_img)
    branch1 = Inceptionv4(X_input_branch1)

    # # Branch 2
    X_input_branch2 = Input(input_shape_dataset)
    branch2 = dense_branch_2(X_input_branch2)

    # # Concatenate branch1 and branch2
    X = tf.concat(values=[branch1, branch2], axis=1)
    X = Dense(1000, activation='relu', name='final_dense_1')(X)
    X = Dense(1000, activation='relu', name='final_dense_2')(X)
    X_out = Dense(1, activation='linear', name='final')(X)

    # Create model
    model = Model(inputs=(X_input_branch1, X_input_branch2), outputs=X_out, name='model0')  #  X_input_branch2
    return model

In [14]:
model = model_assembly(input_shape_img=IMG_SHAPE, input_shape_dataset=1)

model.compile(optimizer="adam", loss="mean_squared_error", metrics=tf.keras.metrics.mean_squared_error)

In [15]:
# Create a callback for early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [16]:
# Callback for tensorboard
log_dir = "./logs/base_fixed_50_epochs"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = "./checkpoints/base_fixed_50_epochs/"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
# Fit the model on batches with real-time data augmentation:
history = model.fit(train_dataset,
                    batch_size=batch_size,
                    validation_data=validation_dataset,
                    epochs=50,
                    # initial_epoch=14,
                    steps_per_epoch=train_steps,
                    validation_steps=validation_steps,
                    callbacks=[tensorboard_callback, cp_callback])

Epoch 1/50
1575/1575 [==============================] - ETA: 0s - loss: 0.1172 - mean_squared_error: 0.1172
Epoch 1: saving model to ./checkpoints/base_fixed_50_epochs\
1575/1575 [==============================] - 479s 295ms/step - loss: 0.1172 - mean_squared_error: 0.1172 - val_loss: 36.5667 - val_mean_squared_error: 36.5667
Epoch 2/50
1575/1575 [==============================] - ETA: 0s - loss: 0.0239 - mean_squared_error: 0.0239
Epoch 2: saving model to ./checkpoints/base_fixed_50_epochs\
1575/1575 [==============================] - 468s 297ms/step - loss: 0.0239 - mean_squared_error: 0.0239 - val_loss: 0.0852 - val_mean_squared_error: 0.0852
Epoch 3/50
1575/1575 [==============================] - ETA: 0s - loss: 0.0187 - mean_squared_error: 0.0187
Epoch 3: saving model to ./checkpoints/base_fixed_50_epochs\
1575/1575 [==============================] - 466s 296ms/step - loss: 0.0187 - mean_squared_error: 0.0187 - val_loss: 0.0203 - val_mean_squared_error: 0.0203
Epoch 4/50
1575/1575

In [21]:
model.save('./models/base_fixed_50_epochs')

INFO:tensorflow:Assets written to: ./models/base_50_epochs\assets


INFO:tensorflow:Assets written to: ./models/base_50_epochs\assets


In [16]:
test_dataset = create_dataset_images(img_paths=X_test,
                                     second_branch_df=second_branch_df_test[['sex']],
                                     ages=age_test_df,
                                     batch_size=batch_size,
                                     shuffle=False,
                                     repeat=False)
second_validation = create_dataset_images(img_paths=X_validation,
                                          second_branch_df=second_branch_df_val[['sex']],
                                          ages=age_validation_df,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          repeat=False)
second_train = create_dataset_images(img_paths=X_train,
                                     second_branch_df=second_branch_df_train[['sex']],
                                     ages=age_train_df,
                                     batch_size=batch_size,
                                     shuffle=False,
                                     repeat=False)

In [17]:
results = model.predict(test_dataset)

25/25 [==============================] - 5s 99ms/step


In [18]:
results_df = pd.DataFrame(results, columns=['Model ouput'])
results_df['Output Age'] = results_df['Model ouput'] * 200
results_df['Real Age'] = age_test_df
results_df['Real Age Months'] = age_test_df * 200
results_df['Error'] = results_df['Model ouput'] - results_df['Real Age']
results_df['Error Months'] = results_df['Error'] * 200
results_df['Absolute Error'] = results_df['Error'].abs()
results_df['Squared Error'] = results_df['Error'] ** 2
results_df

,Model ouput,Output Age,Real Age,Real Age Months,Error,Error Months,Absolute Error,Squared Error
0,0.844615,168.922913,0.840,168.000000,0.004615,0.922918,0.004615,0.000021
1,0.869985,173.997040,0.845,169.000000,0.024985,4.997039,0.024985,0.000624
2,0.385227,77.045471,0.365,73.000000,0.020227,4.045468,0.020227,0.000409
3,0.741997,148.399475,0.760,152.000000,-0.018003,-3.600526,0.018003,0.000324
4,0.714468,142.893677,0.675,135.000000,0.039468,7.893682,0.039468,0.001558
...,...,...,...,...,...,...,...,...
195,0.731814,146.362762,0.665,133.000000,0.066814,13.362753,0.066814,0.004464
196,0.657595,131.519073,0.645,129.000000,0.012595,2.519071,0.012595,0.000159
197,0.815840,163.168091,0.835,167.000000,-0.019159,-3.831899,0.019159,0.000367
198,0.653619,130.723740,0.675,135.000000,-0.021381,-4.276264,0.021381,0.000457


In [19]:
print(np.mean(results_df['Absolute Error']), np.mean(results_df['Absolute Error']) * 200)

0.04524028 9.048055857419968


In [20]:
mse = np.mean((results_df['Absolute Error'] * 200) ** 2)
print(mse, np.sqrt(mse))

197.98154 14.070591


In [36]:
full_model = tf.keras.models.load_model('./models/full_50_epochs')
full_model.summary()

In [40]:
results = full_model.predict(test_dataset)

25/25 [==============================] - 4s 96ms/step
